In [ ]:
#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import sys
sys.executable

import numpy as np

### fixed
#from libsvm.svmutil import *
from libsvm.svmutil import svm_read_problem

import matplotlib.pylab as plt
import sys
import struct
import networkx as nx
import dwave_networkx as dnx
import matplotlib.pyplot as plt
import dimod
import collections
import math
import pickle
import json
from dwave.system import DWaveSampler, EmbeddingComposite, FixedEmbeddingComposite
from dwave.cloud import Client
from dwave.system import DWaveSampler
from dwave.embedding import EmbeddedStructure
import dwave.inspector
import dwave.embedding
from minorminer import find_embedding
import itertools
 
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Function
import math

from torchsummary import summary

import torch

from qubovert import PCBO, boolean_var

# In[ ]:


#! wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a1a
#! wget https://www.csie.ntu.edu.tw/~cjlin/libsvmtools/datasets/binary/a1a.t


# # In[ ]:
# #
# #
# Leap_API_token = '' ### from the leap account dashboard at https://cloud.dwavesys.com/
# DATA = '/home/sasde/Downloads/'

# client = Client.from_config(token=Leap_API_token)
# solvers = client.get_solvers(num_qubits__gt=3000)
# solver = solvers[0]
# solver
# G = nx.Graph()
# G.add_edges_from( solver.edges )




In [2]:
import gurobipy as grb
from gurobipy import GRB


# Data

In [3]:
DATA = './datafolder'

In [4]:
def set_up_data(sel=3):
    
    ## selects a subset of features from the a1a dataset
    
    #global x
    global y
    #global x_test
    global y_test
    global select
    global x_data_reduced
    global x_test_data_reduced
    y, x = svm_read_problem(DATA +'/a1a')
    y_test, x_test = svm_read_problem(DATA +'/a1a.t')
    np.random.seed(2)
    train_indexes = list(range(len(y)))
    np.random.shuffle(train_indexes)
    y = [y[i] for i in train_indexes]
    x = [x[i] for i in train_indexes]
    x_data, exists = data_to_matrix(x)
    x_test_data, exists_test = data_to_matrix(x_test)

    if sel== 0.3:
        select = np.mean(x_data,0) > .3
        np.array(range(123))[select]
    elif sel== 7:     ### selects 7 features
        select = (((np.mean(x_data,0) > .3) + 0.) + (np.mean(x_data,0) < .65)) == 2.
        np.array(range(123))[select]
    elif sel== 9:      ### selects 9 features
        select = (((np.mean(x_data,0) > .3) + 0.) + (np.mean(x_data,0) < .7)) == 2.
        np.array(range(123))[select]

    elif sel== 10:
        select = (((np.mean(x_data,0) > .25) + 0.) + (np.mean(x_data,0) < .74)) == 2.
        np.array(range(123))[select]

    elif sel== 15:      ### selects 15 features
        select = (((np.mean(x_data,0) > .21) + 0.) + (np.mean(x_data,0) < .79)) == 2.
        np.array(range(123))[select]
    elif sel == 5:
        select = (((np.mean(x_data,0) > .34) + 0.) + (np.mean(x_data,0) < .65)) == 2.
        np.array(range(123))[select]
    elif sel == 4:
        select = (((np.mean(x_data,0) > .34) + 0.) + (np.mean(x_data,0) < .46)) == 2.
        np.array(range(123))[select]
    elif sel == 6:
        select = (((np.mean(x_data,0) > .33) + 0.) + (np.mean(x_data,0) < .65)) == 2.
        np.array(range(123))[select]


    elif sel == 2:      ### selects 2 features
        select = np.array([False for i in range(123)])
        select[6] = True
        select[22] = True
        select
    elif sel == 1:      ### selects 1 features
        select = np.array([False for i in range(123)])
        select[22] = True
        select
        
    elif sel == 3:      ### selects 3 features
        select = np.array([False for i in range(123)])
        select[6] = True
        select[22] = True
        select[36] = True
        select
    elif sel== 'all':
        select = np.array([True for i in range(123)])
    else:
        assert False

    x_data_reduced = x_data[:,select]
    x_test_data_reduced = x_test_data[:,select]
    
    
    y = 0+(np.array(y) > 0.)
    y_test = 0+(np.array(y_test) > 0.)

#set_up_data(3)

# BNN

In [5]:

# BNN code adapted from:
# github.com/Akashmathwani/Binarized-Neural-networks-using-pytorch
# 


class BinarizeF(Function):

    @staticmethod
    def forward(cxt, input):
        output = input.new(input.size())
        output[input >= 0] = 1
        output[input < 0] = -1
        #output[input >= 0] = 0.9999
        #output[input < 0] = -0.9999
        return output

    @staticmethod
    def backward(cxt, grad_output):
        grad_input = grad_output.clone()
        return grad_input

# aliases
binarize = BinarizeF.apply

class BinaryTanh(nn.Module):
    def __init__(self):
        super(BinaryTanh, self).__init__()
        #self.hardtanh = nn.Hardtanh() ## during the backward pass the hardtanh is used 
        self.hardtanh = nn.Hardtanh(min_val=-1.001,max_val=1.001) ## during the backward pass the hardtanh is used 
        #self.hardtanh = nn.Tanh() ## during the backward pass the hardtanh is used 
        #self.hardtanh = nn.Identity()
    def forward(self, input):
        #output = self.hardtanh(input/2.1)
        output = self.hardtanh(input)
        output = binarize(output) ## the output is binarized {-1,1} in the forward pass
        
        #if num_classes == 2:     #### CHECK THIS
        #    output += 1
        #    output /= 2
        #    output = output.squeeze()

        #print(output)
        return output
        
class BinaryTanhProx(nn.Module):
    def __init__(self):
        super(BinaryTanhProx, self).__init__()
        #self.hardtanh = nn.Hardtanh() ## during the backward pass the hardtanh is used 
        self.hardtanh = nn.Hardtanh(min_val=-1.001,max_val=1.001) ## during the backward pass the hardtanh is used 
        #self.hardtanh = nn.Tanh() ## during the backward pass the hardtanh is used 
        #self.hardtanh = nn.Identity()
    def forward(self, input):
        #output = self.hardtanh(input/2.1)
        output = self.hardtanh(input)
        
        #if num_classes == 2:     #### CHECK THIS
        #    output += 1
        #    output /= 2
        #    output = output.squeeze()

        #print(output)
        return output

class BinaryLinear(nn.Linear):

    def forward(self, input):
        binary_weight = binarize(self.weight) ## the weights are binarized {-1,1} in the forward pass
        if self.bias is None:
            return F.linear(input, binary_weight)
        else:
            return F.linear(input, binary_weight, self.bias)

    def reset_parameters(self):
        # Glorot initialization
        in_features, out_features = self.weight.size()
        stdv = math.sqrt(1.5 / (in_features + out_features))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.zero_()

        self.weight.lr_scale = 1. / stdv    #### this changes the lr for the weights by a constant factor



# In[ ]:



## Define the NN architecture
class BinaryNet(nn.Module):
    def __init__(self):
        super(BinaryNet, self).__init__()
        self.fc_list = nn.ModuleList()
        if num_classes == 2:
            output_net_size = 1
        else:
            output_net_size = num_classes
        input_layer_size = input_size
        output_layer_size = hidden_n
        
        for i in range(n_layers):
            if i == n_layers -1:
                output_layer_size = output_net_size
            self.fc_list.append( BinaryLinear(input_layer_size, output_layer_size, bias=False) )
            input_layer_size = hidden_n
            
        self.dropout = nn.Dropout(0.2)
        self.activation = BinaryTanh()

    def forward(self, x, return_list=None):
        # flatten image input
        x_ = x.view(-1, input_size)
        # add hidden layer, with relu activation function
        out_list = []
        for fkey, fc_layer in enumerate(self.fc_list):

            x_before = fc_layer(x_)
            if return_list != None:
                #assert isinstance(return_list, list)
                if fkey in return_list or return_list[0] == 'all':
                    x_before.retain_grad()
                    if fkey != 0:
                        #print(fkey)
                        x_.retain_grad()
                    #x.retain_grad()
                    fc_layer.input_features = x_
                    fc_layer.before_activation = x_before ## debug
                    out_list.append(x_before)
            x_ = self.activation(x_before)
            if return_list != None:
                #assert isinstance(return_list, list)
                if fkey in return_list or return_list[0] == 'all':
                    x_.retain_grad()
                    fc_layer.output_features = x_
                    out_list.append(x_)
        if return_list != None:
             return x_, out_list
        return x_

class BinaryLinearProx(nn.Linear):

    def forward(self, input):
        binary_weight = self.weight # binarize(self.weight) ## the weights are binarized {-1,1} in the forward pass
        if self.bias is None:
            return F.linear(input, binary_weight)
        else:
            return F.linear(input, binary_weight, self.bias)

    def reset_parameters(self):
        # Glorot initialization
        in_features, out_features = self.weight.size()
        stdv = math.sqrt(1.5 / (in_features + out_features))
        self.weight.data.uniform_(-stdv, stdv)
        if self.bias is not None:
            self.bias.data.zero_()

        self.weight.lr_scale = 1. / stdv    #### this changes the lr for the weights by a constant factor



# In[ ]:



## Define the NN architecture
class BinaryNetProxQuant(nn.Module):
    def __init__(self):
        super(BinaryNetProxQuant, self).__init__()
        self.fc_list = nn.ModuleList()
        if num_classes == 2:
            output_net_size = 1
        else:
            output_net_size = num_classes
        input_layer_size = input_size
        output_layer_size = hidden_n
        
        for i in range(n_layers):
            if i == n_layers -1:
                output_layer_size = output_net_size
            self.fc_list.append( BinaryLinearProx(input_layer_size, output_layer_size, bias=False) )
            input_layer_size = hidden_n
            
        self.dropout = nn.Dropout(0.2)
        self.activation = BinaryTanhProx()

    def forward(self, x, return_list=None):
        # flatten image input
        x_ = x.view(-1, input_size)
        # add hidden layer, with relu activation function
        out_list = []
        for fkey, fc_layer in enumerate(self.fc_list):

            x_before = fc_layer(x_)
            if return_list != None:
                #assert isinstance(return_list, list)
                if fkey in return_list or return_list[0] == 'all':
                    x_before.retain_grad()
                    if fkey != 0:
                        #print(fkey)
                        x_.retain_grad()
                    #x.retain_grad()
                    fc_layer.input_features = x_
                    fc_layer.before_activation = x_before ## debug
                    out_list.append(x_before)
            x_ = self.activation(x_before)
            if return_list != None:
                #assert isinstance(return_list, list)
                if fkey in return_list or return_list[0] == 'all':
                    x_.retain_grad()
                    fc_layer.output_features = x_
                    out_list.append(x_)
        if return_list != None:
             return x_, out_list
        return x_



# GD or QUBO update

In [6]:

from statistics import *
def set_up_and_optimize_qubo(X, Y, W, print_debug=False, epsilon_zero=0.01):
    
    # using standard vars and quicksum

    Y_numpy = Y.numpy()
    X_numpy = X.numpy()
    grb_model_min_y_swx = grb.Model()
    grb_model_min_y_swx.setParam( 'OutputFlag', False )

    W_binary = [ [ grb_model_min_y_swx.addVar( vtype=GRB.BINARY) for j in range(W.shape[1]) ] for i in range(W.shape[0]) ]
    W_spin = [ [ grb_model_min_y_swx.addVar( vtype=GRB.CONTINUOUS, lb=-2) for j in range(W.shape[1]) ] for i in range(W.shape[0]) ]
    for i  in range(W.shape[0]):
        for j in range(W.shape[1]):
            grb_model_min_y_swx.addConstr( W_spin[i][j] == W_binary[i][j]*2 -1) 
    Z_float = [ [ grb_model_min_y_swx.addVar(lb=-100) for j in range(Y.shape[1]) ] for i in range(Y.shape[0]) ]    #     shape=tuple(Y.shape), vtype=GRB.CONTINUOUS, name="before_activation", lb= -100)
    for i in range(X.shape[0]):
        for j in range(Y.shape[1]):
            # Z_float[i] = W_spin @ x_numpy[i]
            grb_model_min_y_swx.addConstr( grb.quicksum( [ X_numpy[i][k] * W_spin[j][k] for k in range(W.shape[1])] )  == Z_float[i][j] )
    Y_binary = [ [ grb_model_min_y_swx.addVar(vtype=GRB.BINARY) for j in range(Y.shape[1]) ] for i in range(Y.shape[0]) ]
    Y_spin = [ [ grb_model_min_y_swx.addVar(vtype=GRB.CONTINUOUS, lb = -2) for j in range(Y.shape[1]) ] for i in range(Y.shape[0]) ]
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            grb_model_min_y_swx.addConstr( Y_spin[i][j] == Y_binary[i][j]*2 -1 )
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            #grb_model_min_y_swx.addConstr( Z_float[i][j]  * Y_spin[i][j]  >= 0 )  ## matrices 1x1
            grb_model_min_y_swx.addConstr( (Z_float[i][j] + epsilon_zero) * Y_spin[i][j]  >= 0 )  ## matrices 1x1
    Y_diff_ = [ [ grb_model_min_y_swx.addVar(vtype=GRB.CONTINUOUS, lb =-100) for j in range(Y.shape[1]) ] for i in range(Y.shape[0]) ]
    for i in range(Y.shape[0]):
        for j in range(Y.shape[1]):
            grb_model_min_y_swx.addConstr( (Y_spin[i][j] - Y_numpy[i,j])  == Y_diff_[i][j] )  ## matrices 1x1
    grb_model_min_y_swx.setObjective( grb.quicksum( [ Y_diff_[i][j] * Y_diff_[i][j]  for i in range(Y.shape[0]) for j in range(Y.shape[1]) ] ), GRB.MINIMIZE )  ## min \\ Y_diff_ ||_F

    grb_model_min_y_swx.feasRelaxS(0, False, False, True)
    grb_model_min_y_swx.setParam('NonConvex', 2)
    grb_model_min_y_swx.optimize()

    if print_debug:
        print('W_spin')
        print(np.array([[x.X for x in y] for y in W_spin] ))
        print('Z_float')
        print(np.array([[x.X for x in y] for y in Z_float] ))
        #print(Z_float.X)
        print('Y_spin')
        print(np.array([[x.X for x in y] for y in Y_spin] ))
        print('Y_diff_2')
        print(np.array([[x.X for x in y] for y in Y_diff_] ))
        #print(Y_spin.X)
        #print(Y_diff_2.X)
        print('Obj: %g' % grb_model_min_y_swx.objVal)

    #return W_spin
    return np.array([[x.X for x in y] for y in W_spin] )

In [9]:
## old code necessary for old implementation
import importlib.util
spec = importlib.util.spec_from_file_location("BNN_on_QPU_libraries", "./BNN_on_QPU_libraries_dimod_10.py")
BNN_on_QPU_libraries = importlib.util.module_from_spec(spec)
spec.loader.exec_module(BNN_on_QPU_libraries)

BNN_on_QPU_libraries.alpha = 0.
BNN_on_QPU_libraries.Lambda_qubo = 50
BNN_on_QPU_libraries.num_classes = 2            ### binary 



In [ ]:
BNN_on_QPU_libraries.USE_OLD_GREATER_ZERO = False
BNN_on_QPU_libraries.USE_OLD_XNOR = False

In [1]:
def update(model, update_type='sgd', grad_compare=None, epoch = 1):
    
    """
    update_type='sgd'
    
    sgd
    
    update_type='qubo'
    
    argmin_W ( sum_i( |Y_i - sigma(W.X_i)| ) )
    """
    ls = []
    for n_l in range(n_layers):
        W = model.fc_list[n_l].weight      
        if update_type== 'sgd':
            #X.data -= learning_rate*model.fc_list[n_l].weight.grad
            with torch.no_grad():
                #W.copy_( W.data -args.learning_rate*model.fc_list[n_l].weight.grad )
                W.copy_( W.data -args.learning_rate*W.grad )

                t = torch.sign(grad_compare[n_l]).flatten()
                temp = torch.eq(t[t != 0],torch.sign(W.grad).flatten()[t != 0]).sum()
                
                # if temp.item() == 150:
                #     print(torch.sign(W.grad))
                #     print(W_optimized)
                #     print('-------------')
                # ls.append(temp.item() / np.prod(grad_compare[n_l].shape))
                ls.append([temp.item(), (t != 0).sum().item()])
        if update_type == 'proxquant':
            with torch.no_grad():
                W.copy_( W.data -args.learning_rate*W.grad)
                # print(W.grad)
                lam =  0.01 * (epoch + 1)
                # W.copy_(torch.sign(W_theta + torch.sign(W_theta - torch.sign(W_theta)) * torch.nn.functional.relu(torch.abs(W_theta - torch.sign(W_theta))- lam)))
        

                p_sign, p_abs = W.sign(), W.abs()
                a = p_sign + torch.sign(W - p_sign) * torch.relu(torch.abs(W - p_sign) - lam)
                # (F.relu((p_abs - 1).abs() - lam) * (p_abs - 1).sign() + 1)
                print(a)
                W.copy_(a)
                
                t = torch.sign(grad_compare[n_l]).flatten()
                temp = torch.eq(t[t != 0],torch.sign(W.grad).flatten()[t != 0]).sum()
                
                # if temp.item() == 150:
                #     print(torch.sign(W.grad))
                #     print(W_optimized)
                #     print('-------------')
                # ls.append(temp.item() / np.prod(grad_compare[n_l].shape))
                ls.append([temp.item(), (t != 0).sum().item()])
        if update_type== 'signsgd':
            #X.data -= learning_rate*model.fc_list[n_l].weight.grad
            with torch.no_grad():
                #W.copy_( W.data -args.learning_rate*model.fc_list[n_l].weight.grad )
                W.copy_( W.data - args.learning_rate*torch.sign(W.grad) )

                t = torch.sign(grad_compare[n_l]).flatten()
                temp = torch.eq(t[t != 0],torch.sign(W.grad).flatten()[t != 0]).sum()
                
                # if temp.item() == 150:
                #     print(torch.sign(W.grad))
                #     print(W_optimized)
                #     print('-------------')
                # ls.append(temp.item() / np.prod(grad_compare[n_l].shape))
                ls.append([temp.item(), (t != 0).sum().item()])
        elif update_type == 'qubo':
            X = model.fc_list[n_l].input_features.data  /  torch.norm(model.fc_list[n_l].input_features.data, dim=0)**2
            Y = model.fc_list[n_l].output_features.grad
            # print(X.shape)
            # print(Y.shape)
            # print(W.shape)
            # print('==============================')
            # W_ls = []
            # for i in range(X.shape[1]):
            #     Wtemp = W[:,i]
            #     Wtemp = Wtemp[:,None]
            #     Xtemp = X[:,i]
            #     Xtemp = Xtemp[:,None]
            #     W_column = set_up_and_optimize_qubo(Xtemp, Y,Wtemp)
            #     W_ls.append(W_column)
            # W_optimized = np.stack(W_ls)
            W_optimized = set_up_and_optimize_qubo(X, Y, W)
            with torch.no_grad():
                W.copy_( W.data - torch.from_numpy( args.learning_rate*W_optimized ) )
                tempt = grad_compare[n_l]
                tempt = torch.abs(tempt)
                tempt[tempt < 0.01] = 0
                grad_compare[n_l][tempt == 0] = 0
                t = -torch.sign(grad_compare[n_l]).flatten()
    
                # t[t == 0] = 1
                temp = torch.eq(t[t != 0], torch.from_numpy(W_optimized).flatten()[t != 0]).sum()
                # if temp.item() == 150:
                #     print(torch.sign(W.grad))
                #     print(W_optimized)
                #     print('-------------')
                ls.append([temp.item(), (t != 0).sum().item()])
                
        ############ code from BNN_pytorch_new/tmp_2.py
        elif update_type.split('.')[0] == 'old_qubo':
            X = model.fc_list[n_l].input_features      
            Y = - model.fc_list[n_l].output_features.grad
            #W = model.fc_list[n_l].weight

            BNN_on_QPU_libraries.x_data_reduced  = (X.data.numpy() +1.)/2. ## input in binary
            BNN_on_QPU_libraries.selected_to_train = X.shape[0]   ### "batch size"
            BNN_on_QPU_libraries.input_size = X.shape[1]
            BNN_on_QPU_libraries.hidden_n = np.nan               ###
            BNN_on_QPU_libraries.n_layers = 1               ### num layers
            BNN_on_QPU_libraries.num_batches_to_run =  1   ### number of batches to run

            BNN_on_QPU_libraries.model = BNN_on_QPU_libraries.BinaryNet()

            for n_output  in range(  Y.shape[1] ):

                #print('############### #################### ################')

                BNN_on_QPU_libraries.y  = (Y.numpy()[:,n_output] +1. )/2. ## output in binary between zero and one

                #BNN_on_QPU_libraries.output_dimension = Y.shape[1]


                #print(BNN_on_QPU_libraries.model)
                #BNN_on_QPU_libraries.optimization_type = 'grb_hard_c'
                BNN_on_QPU_libraries.optimization_type = update_type.split('.')[1]
                #BNN_on_QPU_libraries.optimization_type = 'sym_anneal_soft_qubo'
                BNN_on_QPU_libraries.train_models(layers_to_optimize=[0])
                BNN_on_QPU_libraries.list_results_grb_hard_c = BNN_on_QPU_libraries.list_results

                layer_i = 0
                _y = 0
                for _x in range(X.shape[1]):
                    W_solution = BNN_on_QPU_libraries.H_solution[f'weight_{layer_i}_{_y}_{_x}']
                    W_solution = W_solution*2-1 #convert to spin
                    W.grad[n_output,_x] = W_solution

                W.data += args.learning_rate*W.grad

        ############
                
        
                
        elif update_type == 'compare':
            with torch.no_grad():
                #W.copy_( W.data -args.learning_rate*model.fc_list[n_l].weight.grad )
                W.copy_( W.data -args.learning_rate*W.grad )
            X = model.fc_list[n_l].input_features.data      
            Y = - model.fc_list[n_l].output_features.grad
            plt.imshow( W.grad.numpy() )
            plt.show()
            W_optimized = set_up_and_optimize_qubo(X, Y, W)
            plt.imshow( W_optimized )   
            plt.show()
            plt.figure()
            plt.show()

        elif update_type == 'compare_gt_grad':
            with torch.no_grad():
                #W.copy_( W.data -args.learning_rate*model.fc_list[n_l].weight.grad )
                W.copy_( W.data -args.learning_rate*W.grad )
            X = model.fc_list[n_l].input_features.data      
            Y = - model.fc_list[n_l].output_features.grad
            #plt.imshow( W.grad.numpy() )
            #plt.show()
            W_optimized = set_up_and_optimize_qubo(X, Y, W)
            plt.imshow( model.fc_list[n_l].output_features.data.T )
            plt.title( 'Y_data' )
            plt.show()
            plt.imshow( Y.T )
            plt.title('Y')
            plt.show()
            plt.figure()
            plt.show()
        return ls
        return mean(ls)



# set up data

In [12]:

def data_to_matrix(x):
    x_data = np.zeros((len(x),123))
    exists = np.zeros(123)
    for i,x_i in enumerate( x ):
        for j in range(123):
            if j in x_i.keys():
                assert x_i[j] == 1
                x_data[i,j] = 1.
                exists[j] = 1.
    return x_data, exists


In [ ]:
import time

def timeSince(since):
    now = time.time()
    s = now - since
    #m = math.floor(s / 60)
    #s -= m * 60
    return s

# BNN code adapted from:
# github.com/Akashmathwani/Binarized-Neural-networks-using-pytorch

def test_loss():
    global logs
    data = x_test_data_reduced
    target = y_test
    data = data.astype('float32')
    target = 1.*target
    data = torch.from_numpy(data)
    target = torch.from_numpy(target)
    data, target = torch.autograd.Variable(data), torch.autograd.Variable(target)
    #optimizer.zero_grad()
    model.zero_grad()
    output = model(data)
    if num_classes == 2:
        output = output.T[0]
    loss = criterion(output, target)
    # acc = metrics.roc_auc_score(output.detach().numpy(), target.detach().numpy())
    # print(acc)
    #return float(loss.data.detach().numpy())
    logs['testloss'].append(float(loss.data.detach().numpy()))

def train_nn(update_type='sgd', test_every_step=False, run_id = 0):
    global logs
    #if epoch == 1:
    #    from IPython import embed; embed()
    data_grad = x_data_reduced
    data_grad = data_grad.astype('float32')
    data_grad = torch.from_numpy(data_grad)
    data_grad = data_grad*2 -1 ## convert to spin
    target_grad = y
    target_grad = torch.from_numpy(target_grad)

    target_grad = 1.*target_grad
    target_grad, target_grad = torch.autograd.Variable(target_grad), torch.autograd.Variable(target_grad)
    for batch_n in range(num_batches_to_run):
        
        #if GRADIENT_DESCENT:
        #    batch_n = 0
            
        #print(batch_n)

        #print([i for i in  model.parameters()][0].grad)

        #for index_i in range(selected_to_train):
        #    in_x = x_data_reduced[batch_n*selected_to_train+index_i]
        #    #in_x = 0+(in_x > 0.)
        #    ( in_x, y[batch_n*selected_to_train+index_i])
        data = x_data_reduced[batch_n*selected_to_train: (batch_n+1)*selected_to_train]
        target = y[batch_n*selected_to_train: (batch_n+1)*selected_to_train]
        #from IPython import embed; embed()

        #if args.cuda:
        #    data, target = data.cuda(), target.cuda()

        data = data.astype('float32')
        
        data = data*2 -1 ## convert to spin

        #target = (target+1)/2. ## convert to spins
        #target = (target*2.) -1. ## convert to spins
        target = 1.*target
        
        if update_type == 'compare_gt_grad':
            plt.imshow(np.array([target]))
            plt.show()

        data = torch.from_numpy(data)
        target = torch.from_numpy(target)

        data, target = torch.autograd.Variable(data), torch.autograd.Variable(target)
        #optimizer.zero_grad()
        model.zero_grad()
        output_grad, _ = model(data_grad, return_list=['all'])
        if num_classes == 2:
            output_grad = output_grad.T[0]

        #from IPython import embed; embed()
        loss = criterion(output_grad, target_grad)
        loss.backward()
        W_grad = []
        W_grad.append(model.fc_list[0].weight.grad)
        # W_grad.append(model.fc_list[1].weight.grad)


        model.zero_grad(set_to_none = True)
        #output = model(data)
        output, x_list = model(data, return_list=['all'])

        ## output is a spin

        #if epoch == 1 and batch_n == 0:
        #    from IPython import embed; embed()

        if num_classes == 2:
            output = output.T[0]

        #from IPython import embed; embed()
        loss = criterion(output, target)
        # acc = metrics.roc_auc_score(output.detach().numpy(), target.detach().numpy())
        # print(acc)
        #if epoch == 9 and batch_n == 0:
        #    from IPython import embed; embed()
        #from IPython import embed; embed()

        #optimizer.zero_grad()
        model.zero_grad()
        loss.backward()
        
        #for p in list(model.parameters()):
        #    if hasattr(p,'org'):
        #        print('ORG')
        #        print(p)
        #        p.data.copy_(p.org)
        

        diff = update(model, update_type = update_type, grad_compare = W_grad)
        # logs['diff'].append(diff[0][0])
        # logs['diff_sum'].append(diff[0][1])

        #optimizer.step()
        
        #for p in list(model.parameters()):
        #    if hasattr(p,'org'):
        #        print('ORG')
        #        print(p)
        #        p.org.copy_(p.data.clamp_(-1,1))
        
        #from IPython import embed; embed()
        #print(batch_n,loss.data.numpy())
        #plt.plot([batch_n], [loss.data.numpy()], '.')
        
        #if epoch == 9 and batch_n == 0:
        #    from IPython import embed; embed()

        #if batch_idx % args.log_interval == 0:
        if batch_n % args.log_interval == 0:
            logs['tlos'].append(float(loss.data.detach().numpy()))
            # with open(f"/home/mkrahn/quantum_g/quantum_gnn/to_tolga_incremental_training_BNN/loss_{run_id}.csv", "a") as f:
            #     f.write(loss.data.detach().numpy())
            #     f.write(f"\n")
            if test_every_step:
                test_loss()
        #    print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}'.format(
        #        epoch, batch_idx * len(data), len(train_loader.dataset),
        #        100. * batch_idx / len(train_loader), loss.data[0]))


#def adjust_learning_rate(optimizer, epoch):
def adjust_learning_rate(epoch):
    update_list = [55, 100, 150,200,400,600]
    if epoch in update_list:
        #for param_group in optimizer.param_groups:
        #    param_group['lr'] = param_group['lr'] * 0.1
        args.learning_rate = args.learning_rate *0.1
    return



def Train(update_type='sgd', test_every_step=False, run_id=0):
    global logs
    start = time.time()
    logs = {}
    logs['time_graph'] = []
    logs['e']=[]
    logs['accur']=[]
    logs['tlos']=[]
    logs['testloss']=[]
    logs['lr'] = []
    logs['diff'] = []
    logs['diff_sum'] = []


    for epoch in range(1, args.epochs + 1):
        #print('epoch: ', epoch)
        #adjust_learning_rate(optimizer, epoch)
        adjust_learning_rate(epoch)    
        logs['e'].append(epoch)
        if not test_every_step:
            test_loss() 
        train_nn(update_type=update_type, test_every_step=test_every_step, run_id= run_id)   
        seco=timeSince(start)
        logs['time_graph'].append(seco)
        #logs['lr'].append(optimizer.param_groups[0]['lr'])
        logs['lr'].append(args.learning_rate)
        #test()

def plots():
    #print(time_graph)
    plt.title('Training with epoch', fontsize=20)
    plt.ylabel('time (s)')
    plt.plot(logs['e'],logs['time_graph'])
    plt.show()
    #plt.title('Accuracy With epoch', fontsize=20)
    #plt.plot(logs['e'],logs['accur'])
    #plt.show()
    plt.title('Train loss', fontsize=20)
    plt.plot(logs['tlos'])
    plt.show()
    plt.title('Test loss', fontsize=20)
    plt.plot(logs['testloss'])
    plt.show()
    plt.title('lr', fontsize=20)
    plt.plot(logs['lr'])
    plt.show()

